In [ ]:
from bs4 import BeautifulSoup
import requests
import re
import collections
import pickle

In [ ]:
# init dict
Data = collections.defaultdict(list)

# Wikipedia

In [ ]:
# URL for database
#URL = 'https://en.wikipedia.org/wiki/List_of_birds_of_the_Netherlands'
#URL = 'https://en.wikipedia.org/wiki/List_of_mammals_of_the_Netherlands'
URL = 'https://en.wikipedia.org/wiki/List_of_mammals_of_Europe'
page = requests.get(URL)
soup = BeautifulSoup(page.content, 'html.parser')

# Find all wikiparts
Animals = soup.find_all('a')
# Create links 
AnimalsWikiPages = ['https://en.wikipedia.org/' + pages.get('href') for pages in Animals 
                       if pages.get('href') != None 
                       if pages.get('href').startswith('/wiki/')]
                       # Reduces the retrieved pages (does not work)
                       #if pages.span != None 
                       #if pages.span.attrs['class'][0] == 'tocnumber']

In [ ]:
# init dicts
DescriptionData = {}
NonDescriptionData = {}

# Removes references in text
ReferenceRemover = '\[\d*\]'

for WikiPage in AnimalsWikiPages[:]:
    
    # Open the page
    page = requests.get(WikiPage, timeout=5)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    for Tags in soup.find_all('h2'):

        # Skip useless/empty stuff
        if Tags.span == None:
            continue

        # Set chapter variable    
        Chapter = Tags.span.attrs['id']

        # Check if the chapter is description (or similar)
        if Chapter == 'Characteristics'or \
           Chapter == 'Description' or \
           Chapter == 'Appearance':


            # Get species name
            Species = soup.title\
                            .string\
                            .split(' - ')[0]\
                            .rstrip(' ')


            # Get the next sibling (text)
            for Text in Tags.find_next_siblings('p'):

                # Add description data to dict
                if Chapter in Text.find_previous_siblings('h2')[0].text.strip():
                    # Remove source
                    Paragraph = re.sub(ReferenceRemover, '', Text.text)
                    # Split into Sentences
                    SentenceList = Paragraph.split('. ')
                    # Add to the dict
                    Data[Species] += [(1, Sentence) for Sentence in SentenceList]

                # Add non description data to dict
                elif Chapter not in Text.find_previous_siblings('h2')[0].text.strip():
                    # Remove source
                    Paragraph = re.sub(ReferenceRemover, '', Text.text)
                    # Split into Sentences
                    SentenceList = Paragraph.split('. ')
                    # Add to the dict
                    Data[Species] += [(0, Sentence) for Sentence in SentenceList]

In [ ]:
len(Data.keys())

In [ ]:
with open('data.pkl', 'wb') as f:
    pickle.dump(Data, f)

# Animals A-Z

In [ ]:
URL = 'https://a-z-animals.com/animals/'
page = requests.get(URL)
soup = BeautifulSoup(page.content, 'html.parser')

# Finds all links
Animals = soup.find_all('a')

# Create a list with links
AnimalsA_Zpages = [pages.get('href') for pages in Animals 
                   if pages.get('href') != None
                   if pages.get('href').startswith('https://a-z-animals.com')]

In [ ]:
# Removes references in text
ReferenceRemover = '\[\d*\]'

for AnimalPage in AnimalsA_Zpages:
    
    # Open the page
    page = requests.get(AnimalPage, timeout=5)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    for Tags in soup.find_all('h2'):

        # Get the chapters
        try:
            Chapter = Tags['id']
        # skip other stuff
        except:
            continue

        # Finds descriptions or similar
        if Chapter.endswith('appearance'):
            
            '''
            # Get species name
            Species = soup.title\
                            .string\
                            .split(' Animal')[0]\
                            .lower()\
                            .capitalize()
            '''
            
            # Get species name
            Species = soup.find('h1').text\
                                        .lower()\
                                        .capitalize()


            # Get the next sibling (text)
            for Text in Tags.find_next_siblings('p'):

                # Add description data to dict
                if Chapter in Text.find_previous_siblings('h2')[0]['id']:
                    # Remove source
                    Paragraph = re.sub(ReferenceRemover, '', Text.text)
                    # Split into Sentences
                    SentenceList = Paragraph.split('. ')
                    # Add to the dict
                    Data[Species] += [(1, Sentence) for Sentence in SentenceList]

                # Add non description data to dict
                elif Chapter not in Text.find_previous_siblings('h2')[0]['id']:
                    # Remove source
                    Paragraph = re.sub(ReferenceRemover, '', Text.text)
                    # Split into Sentences
                    SentenceList = Paragraph.split('. ')
                    # Add to the dict
                    Data[Species] += [(0, Sentence) for Sentence in SentenceList]
        

In [ ]:
with open('data.pkl', 'wb') as f:
    pickle.dump(Data, f)

In [ ]:
# Total values
Total = sum(len(v) for v in list(Data.values()))
print(Total)
# Total Truths
TotalValues = [v for v in list(Data.values())]
Truths = [[ones[0] for ones in Values if ones[0] == 1] for Values in TotalValues]
AmountofTruths = sum(len(x) for x in Truths)
AmountofFalse = Total - AmountofTruths
print(AmountofTruths)
print(AmountofFalse)

# Birds of the World

In [ ]:
URL = 'https://birdsoftheworld.org/bow/specieslist'
#URL = 'https://birdsoftheworld-org.ezproxy.library.wur.nl/bow/specieslist'
page = requests.get(URL)
soup = BeautifulSoup(page.content, 'html.parser')

BOWlist = soup.find_all('a')

URL = 'https://birdsoftheworld.org/bow/specieslist'
#URL = 'https://birdsoftheworld-org.ezproxy.library.wur.nl/bow/specieslist'
page = requests.get(URL)
soup = BeautifulSoup(page.content, 'html.parser')

# Create a list with links
BOWpages = ['https://birdsoftheworld.org' + pages.get('href').replace('introduction', '') for pages in BOWlist 
                   if pages.get('href') != None
                   if pages.get('href').startswith('/bow/species')]

In [ ]:
BOWpages[3333]

In [ ]:
BOWpages.index('https://birdsoftheworld.org/bow/species/norcar/cur/')

In [ ]:
#URL = BOWpages[10566] + 'appearance'
URL = 'https://birdsoftheworld.org/bow/species/anteup1/cur/introduction'
page = requests.get(URL)
soup = BeautifulSoup(page.content, 'html.parser')

In [ ]:

# Start the session
session = requests.Session()

# Create the payload
payload = {'username':'RobertvdV', 
           'password':'XXXXXXXXX',
         }

agent = {"User-Agent": 'Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.115 Safari/537.36'}

# Load the page
page = session.get("https://birdsoftheworld.org/bow/species/citwar1/cur/introduction?login")

# Post the payload to the site to log in
page = session.post("https://secure.birds.cornell.edu/cassso/login?service=https%3A%2F%2Fbirdsoftheworld.org%2Flogin%2Fcas", 
                    data=payload,
                    headers=agent)


# Navigate to the next page and scrape the data
page = session.get('https://birdsoftheworld.org/bow/species/citwar1/cur/introduction?login')

session.close()

soup = BeautifulSoup(page.content, 'html.parser')


In [ ]:
soup.title

In [ ]:
session.cookies

In [ ]:
# Init dict
DataBOW = collections.defaultdict(list)

# Reference Remover
ReferenceRemover1 = ' \(\d+\n\t.+?\n\t\n\t\tClose\n\t\n\)'
ReferenceRemover2 = ' \[\d+\n\t.+?\n\t\n\t\tClose\n\t\n\]'
# Brackets Remover
BracketsRemover = ' \((.*?)\)'

# Bird name 
BirdName = soup.title\
                .text\
                .split(' - ')[1]\
                .lstrip()

# Find all text data
Paragraphs = soup.find_all('p')

# Loop over the text
for Para in Paragraphs[1:]:
    try:
        
        # Check to which section the text belongs
        SubSection = Para.find_previous_siblings()[0]\
                         .text\
                         .rstrip()\
                         .lstrip()
        
        # Check to which section the text belongs
        SubSubSection = Para.find_previous_siblings()[-1]\
                             .text\
                             .rstrip()\
                             .lstrip()
        
    # If a text does not have a header, skip    
    except:
        continue
    
    if 'Formative' in SubSection:
        continue
    if 'Juvenile' in SubSection:
        continue
    
    # See which are needed and append with 1
    if SubSection == 'Identification' or\
       SubSection == 'Adult' or\
       SubSubSection == 'Plumages' or\
       SubSubSection == 'Bare Parts':
          
        # Clean the text from references (NEEDS EDIT)    
        TextCleaned = re.sub(ReferenceRemover1, '', Para.text, re.DOTALL)
        TextCleaned = re.sub(ReferenceRemover2, '', TextCleaned, re.DOTALL)
        TextCleaned = re.sub(BracketsRemover, '', TextCleaned)
        
        # Drop useless short nonsense
        if len(TextCleaned.split()) <2:
            continue
        else:
            DataBOW[BirdName] += (1, TextCleaned)
            
    # Otherwise label 2        
    else:
        # Clean the text from references (NEEDS EDIT)    
        TextCleaned = re.sub(ReferenceRemover1, '', Para.text, re.DOTALL)
        TextCleaned = re.sub(ReferenceRemover2, '', TextCleaned, re.DOTALL)
        TextCleaned = re.sub(BracketsRemover, '', TextCleaned)
        
        # Drop useless short nonsense
        if len(TextCleaned.split()) <2:
            continue
        else:
            DataBOW[BirdName] += (0, TextCleaned)

In [ ]:
DataBOW[BirdName]

In [ ]:
'''
# Start the session
session = requests.Session()

# Create the payload
payload = {'UserName':'robert.vandevlasakker@wur.nl', 
           'Password':'Hints4-hung###'
         }

agent = {"User-Agent": 'Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.115 Safari/537.36'}

# Load the WUR library page
#WURpage = session.get('https://login.wur.nl/adfs/ls/?SAMLRequest=lZJBb8IwDIX%2FSpU7TWkZpRFUYnAYEhsIuh12C6mBSGnSxclg%2F35Z2TR2QdrV8ffs95wx8ka1bOrdUW%2FgzQO66Nwojax7mBBvNTMcJTLNG0DmBNtOH5csjRPWWuOMMIpcIbcJjgjWSaNJtJhPyGy1qVaCZ3dpzYf9NC%2BKJMt5XQ%2Bzuj%2FKinq33w1EkXFIBmJIohewGNgJCVJBANHDQqPj2oVSkvZ7Sd5LRlWSszRlWf%2BVRPPgR2ruOuroXIuMUmUOUscnb2OtKK%2F3SBVSEk1%2FdpsZjb4BuwX7LgU8b5a%2FLOjAQoze7oXRcHadRkgPtJOiG0SxpeIi0bvyu%2F4O617qWurD7Zx2lyZkD1W17q1X24qU4694Wefalv%2FcpwHHa%2B74mF6LjC%2FHfwrjF%2FO1UVJ8RFOlzGlmgTuYEGc9EFpeqL%2B%2FpPwE')

# Load a WUR login sessions
# Post the payload to the site to log in
login = session.post("https://login.wur.nl/adfs/ls/?SAMLRequest=lZJBb8IwDIX%2FSpV7SRtGKVGLxOAwJDYQdDvsFlIXIqUJi9PB%2Fv26dtPYBWlXx9%2Bz33MyFLU%2B8Vnjj2YLbw2gDy61Nsi7h5w0znArUCE3ogbkXvLd7HHF2SDiJ2e9lVaTK%2BQ2IRDBeWUNCZaLnMzX22KdVHvGhqlIRRRPYpZE8UiyskqTSRWVd%2Fs4TRmbJHJIghdw2LI5aaVaAcQGlga9ML4tRSwOo3EYpUU05mzEWfJKgkXrRxnhO%2Bro%2FQk5pdoelBmcGzcwmoqyQqqRkmD2s9vcGmxqcDtw70rC83b1y4JpWRhg4yppDVx8p9GmB8Yr2Q2ieKKylwiv%2FG6%2Bw7pXplTmcDunfd%2BE%2FKEoNuFmvSvINPuKl3eu3fSf%2B9TgRSm8yOi1SNYf%2F6kdv1xsrFbyI5hpbc9zB8JDTrxrgNBpT%2F39JdNP", 
                    data=payload,
                    headers=agent)


# Navigate to the next page and scrape the data
#page = session.get('https://birdsoftheworld-org.ezproxy.library.wur.nl/bow/home')
page = session.get('https://birdsoftheworld-org.ezproxy.library.wur.nl/bow/species/anteup1/cur/appearance')

login = session.post("https://login.wur.nl/adfs/ls/?SAMLRequest=lZJBb8IwDIX%2FSpV7SRtGKVGLxOAwJDYQdDvsFlIXIqUJi9PB%2Fv26dtPYBWlXx9%2Bz33MyFLU%2B8Vnjj2YLbw2gDy61Nsi7h5w0znArUCE3ogbkXvLd7HHF2SDiJ2e9lVaTK%2BQ2IRDBeWUNCZaLnMzX22KdVHvGhqlIRRRPYpZE8UiyskqTSRWVd%2Fs4TRmbJHJIghdw2LI5aaVaAcQGlga9ML4tRSwOo3EYpUU05mzEWfJKgkXrRxnhO%2Bro%2FQk5pdoelBmcGzcwmoqyQqqRkmD2s9vcGmxqcDtw70rC83b1y4JpWRhg4yppDVx8p9GmB8Yr2Q2ieKKylwiv%2FG6%2Bw7pXplTmcDunfd%2BE%2FKEoNuFmvSvINPuKl3eu3fSf%2B9TgRSm8yOi1SNYf%2F6kdv1xsrFbyI5hpbc9zB8JDTrxrgNBpT%2F39JdNP", 
                    data=payload,
                    headers=agent)

page = session.get('https://birdsoftheworld-org.ezproxy.library.wur.nl/bow/species/anteup1/cur/appearance')


soup = BeautifulSoup(page.content, 'html.parser')
'''

In [ ]:
soup.title